In [1]:
import psycopg2
from osgeo import ogr
from data import acces

In [2]:
connection = psycopg2.connect(dbname=acces.DATABASE_NAME,
                             user=acces.USER_NAME,
                             password=acces.PASS,
                             host='localhost',
                             port='5432')

In [4]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("DROP TABLE IF EXISTS borders")
        cursor.execute("CREATE TABLE borders (" +
                       "id SERIAL PRIMARY KEY," +
                       "name VARCHAR NOT NULL," +
                       "iso_code VARCHAR NOT NULL," +
                       "outline GEOGRAPHY)")
        cursor.execute("CREATE INDEX border_index ON borders " +
                       "USING GIST(outline)")
        connection.commit()

In [6]:
shapefile = ogr.Open('input/TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp')
layer = shapefile.GetLayer(0)

In [8]:
for i in range(layer.GetFeatureCount()):
    feature = layer.GetFeature(i)
    name = feature.GetField('NAME')
    iso_code = feature.GetField('ISO3')
    geometry = feature.GetGeometryRef()
    wkt = geometry.ExportToWkt()
    with connection:
        with connection.cursor() as cursor:
            cursor.execute("INSERT INTO borders (name, iso_code, outline) " +
                           "VALUES (%s, %s, ST_GeogFromText(%s))",
                           (name, iso_code, wkt))
        connection.commit()

In [10]:
start_long = 8.542
start_lat = 47.377
radius = 500000

In [11]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("SELECT name FROM borders WHERE ST_DWithin(" +
                       "ST_MakePoint(%s, %s), outline, %s)",
                       (start_long, start_lat, radius))
        for row in cursor:
            print(row[0])

Switzerland
Austria
Czech Republic
France
Germany
Croatia
Italy
Liechtenstein
Belgium
Luxembourg
Monaco
Netherlands
Slovenia
San Marino


In [12]:
# обновляем оптимизатор запросов
with connection:
    with connection.cursor() as cursor:
        old_level = connection.isolation_level
        connection.set_isolation_level(0)
        cursor.execute("VACUUM ANALYZE") 
        connection.set_isolation_level(old_level)

In [13]:
# всегда надо закрывать коннект с базой (он не закрывается в менеджере контекста)
connection.close()